In [3]:
from flask import Flask, render_template, request
import numpy as np
import pandas as pd
import base64
import io 
import base64
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from qiskit import Aer
from qiskit import transpile, execute
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import SPSA
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.exceptions import QiskitMachineLearningError
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit.visualization import circuit_drawer
import time



app = Flask(__name__,static_folder='static')

@app.route('/',methods=['GET', 'POST'])
def index():

    if request.method == 'POST':
        
        model = request.form['model']
        dataset = request.form['dataset']
        feature_map_type = request.form['featuremap']
        optimizer_type = request.form['optimizer']
        train_test = request.form['train_test']
        pca = request.form['pca']
        entang = request.form['entang']
        
        
        
        
        X,y= load_dataset(dataset)
        
        X = pca(pca, X)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        X_train_scaled, X_test_scaled = preprocess_data(X_train, X_test)
        
        #df = pd.DataFrame(X_test_scaled)
        
        df
       
        feature_map = feature_map(feature_map_type, entag, X.shape[1])
       
        #ansatz = RealAmplitudes(num_qubits=X.shape[1], reps=3)
        #optimizer = create_optimizer(optimizer_type)      
        #print(X_train_scaled)   
        return render_template('dashboard.html',feature_map=feature_map,dataset=dataset,optimizer_type=optimizer_type)
        # Handle the GET request and render the form
    return render_template('dashboard.html')
    

def load_dataset(dataset):
    if dataset == 'iris':
        iris = datasets.load_iris()
        X = iris.data
        y = iris.target
        return X, y
    if dataset == 'diabetes':
        diabetes = datasets.load_diabetes()
        X = diabetes.data
        y = diabetes.target
        return X, y
    if dataset == 'wine':
        wine = datasets.load_wine()
        X = wine.data
        y = wine.target
        return X, y
    if dataset == 'cancer':
        cancer = datasets.load_breast_cancer()
        X = cancer.data
        y = cancer.target
        return X, y

def preprocess_data(X_train, X_test):
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

def feature_map(feature_map_type, feature_dimension):
    if feature_map_type == 'zz':
        image_stream = io.BytesIO()
        feature_map = ZZFeatureMap(feature_dimension=feature_dimension, reps=1)
        feature_map.decompose().draw(output="mpl", fold=20,filename=image_stream  )
        image_stream.seek(0)
        base64_image = base64.b64encode(image_stream.getvalue()).decode('utf-8')
        return base64_image, feature_map
    elif feature_map_type == 'z':
        image_stream = io.BytesIO()
        feature_map = ZFeatureMap(feature_dimension=feature_dimension, reps=1)
        feature_map.decompose().draw(output="mpl", fold=20,filename=image_stream  )
        image_stream.seek(0)
        base64_image = base64.b64encode(image_stream.getvalue()).decode('utf-8')
        return base64_image
    
    elif feature_map_type == 'pauli':
        return PauliFeatureMap(feature_dimension=feature_dimension, reps=1)

def optimizer(optimizer_type):
    if optimizer_type == 'spsa':
        return SPSA(maxiter=100)
    if optimizer_type == 'coby':
        return COBYLA(maxiter=100)
    if optimizer_type == 'adam':
        return COBYLA(maxiter=100)
    
def pca(pca_no,X):
    pca = decomposition.PCA(n_components=pca_no)
    pca.fit(X)
    X = pca.transform(X)
    return X

def load_model(model, X_train_scaled, X_test_scaled, y_train, y_test,optimizer,feature_map,ansatz  ):
    if model == "MLP/VQC":
        c_metrics = mlp_classifier(X_train_scaled, X_test_scaled)
        q_metrics = vqc_classifier(optimizer,feature_map,ansatz, X_train_scaled, X_test_scaled, y_train, y_test)
        bchart = barchart_mlp_vqc(c_metrics,q_metrics)
        return bchart

#=============================== MLP / VQC CLASSIFIER==========================

def mlp_classifer( X_train_scaled, X_test_scaled, y_train, y_test):
    # Create an instance of the MLP classifier
    mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=5000, random_state=42)
    # Train the MLP classifier
    mlp.fit(X_train_scaled, y_train_scaled)
    # Make predictions with MLP classifier
    y_pred_mlp = mlp.predict(X_test_scaled)
    # Calculate accuracy, F1 score, precision, and recall for MLP classifier
    accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
    f1_mlp = f1_score(y_test, y_pred_mlp, average='weighted')
    precision_mlp = precision_score(y_test, y_pred_mlp, average='weighted')
    recall_mlp = recall_score(y_test, y_pred_mlp, average='weighted')    
    return accuracy_mlp,f1_mlp,precision_mlp, recall_mlp

objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()
    
def vqc_classifier(optimizer,feature_map,ansatz, X_train_scaled, X_test_scaled, y_train, y_test):
    optimizer = COBYLA(maxiter=100)
    sampler = Sampler()
    vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    callback=callback_graph,
    sampler=sampler
    )
    vqc.fit(X_train, y_train)
    # Make predictions with VQC
    y_pred_vqc = vqc.predict(X_test_scaled)
    # Calculate accuracy, F1 score, precision, and recall for VQC
    accuracy_vqc = accuracy_score(y_test, y_pred_vqc)
    f1_vqc = f1_score(y_test, y_pred_vqc, average='weighted')
    precision_vqc = precision_score(y_test, y_pred_vqc, average='weighted')
    recall_vqc = recall_score(y_test, y_pred_vqc, average='weighted')
    return accuracy_vqc,f1_vqc,precision_vqc, recall_vqc

def barchart_mlp_vqc():
    # Bar chart
    labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    svm_scores = [accuracy_mlp, precision_mlp, recall_mlp, f1_mlp]
    qsvm_scores = [accuracy_vqc, precision_vqc, recall_vqc, f1_vqc]

    x = np.arange(len(labels))
    width = 0.35

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, svm_scores, width, label='MLP')
    rects2 = ax.bar(x + width/2, qsvm_scores, width, label='VQC')

    ax.set_ylabel('Scores')
    ax.set_title('Comparison of Metrics: MLP vs VQC')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    fig.tight_layout()

    plt.show()
    
   
  
#================================= QSVM/SVM======================================================

def svm_classifier():
    svm = svm.SVC(kernel='linear')
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    
    svm_accuracy = accuracy_score(y_test, svm_pred)
    svm_precision = precision_score(y_test, svm_pred, average='macro')
    svm_recall = recall_score(y_test, svm_pred, average='macro')
    svm_f1 = f1_score(y_test, svm_pred, average='macro')
    return svm_accuracy,svm_precision, svm_recall,svm_f1
    
    
def qsvm_classifier():
    
    qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=Aer.get_backend('qasm_simulator'))
    qsvm = QSVC(quantum_kernel=qkernel)
    start = time.time()
    epochs = 10
    for _ in tqdm(range(epochs), desc="Training Progress"):
        qsvm.fit(X_train, y_train)
    elapsed = time.time() - start
    # Predict labels for the test set
    y_pred = qsvm.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    qsvm_accuracy = np.sum(y_pred == y_test) / len(y_test)
    qsvm_f1 = f1_score(y_test, y_pred, average='weighted')
    qsvm_precision = precision_score(y_test, y_pred, average='weighted')
    qsvm_recall = recall_score(y_test, y_pred, average='weighted')
    
    return qsvm_accuracy,qsvm_f1,qsvm_precision,qsvm_recall
    
def barchart_qsvm_svm():
    # Bar chart
    labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    svm_scores = [svm_accuracy, svm_precision, svm_recall, svm_f1]
    qsvm_scores = [qsvm_accuracy, qsvm_precision, qsvm_recall, qsvm_f1]

    x = np.arange(len(labels))
    width = 0.35

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, svm_scores, width, label='SVM')
    rects2 = ax.bar(x + width/2, qsvm_scores, width, label='QSVM')

    ax.set_ylabel('Scores')
    ax.set_title('Comparison of Metrics: SVM vs QSVM')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    fig.tight_layout()

    plt.show()
    
#================================QSVC/SVC======================================================

def svc_classifier():
    svc = SVC()
    svc.fit(X_train, y_train) 
    svc_pred = svc.predict(X_test)
    
    svc_accuracy = accuracy_score(y_test, svc_pred)
    svc_precision = precision_score(y_test, svc_pred, average='macro')
    svc_recall = recall_score(y_test, svc_pred, average='macro')
    svc_f1 = f1_score(y_test, svc_pred, average='macro')
    return svc_accuracy,svc_precision, svc_recall,svc_f1

def qsvc_classifier():
    sampler = Sampler()
    fidelity = ComputeUncompute(sampler=sampler)
    kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    qsvc = QSVC(quantum_kernel=kernel)
    start = time.time()
    qsvc.fit(X_train, y_train)
    elapsed = time.time() - start
    y_pred = qsvc.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    qsvc_accuracy = np.sum(y_pred == y_test) / len(y_test)
    qsvc_f1 = f1_score(y_test, y_pred, average='weighted')
    qsvc_precision = precision_score(y_test, y_pred, average='weighted')
    qsvc_recall = recall_score(y_test, y_pred, average='weighted')    
    return qsvc_accuracy,qsvc_f1,qsvc_precision,qsvc_recall


#================================VQC/LR======================================================


    
def lr_classifier():
    # Create a logistic regression CL_model_LR
    CL_model_LR = LogisticRegression()
    # Train the CL_model_LR
    CL_model_LR.fit(train_features, train_labels)
    # Make predictions on the test set
    y_pred_model1 = CL_model_LR.predict(test_features)
    svc_accuracy = accuracy_score(y_test, svc_pred)
    svc_precision = precision_score(y_test, svc_pred, average='macro')
    svc_recall = recall_score(y_test, svc_pred, average='macro')
    svc_f1 = f1_score(y_test, svc_pred, average='macro')
    return svc_accuracy,svc_precision, svc_recall,svc_f1


objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

def ansatz_cir(N):
    image_stream = io.BytesIO()
    ansatz = RealAmplitudes(num_qubits=N, reps=3)
    ansatz.decompose().draw(output="mpl", fold=20,filename=image_stream )
    image_stream.seek(0)
    base64_image = base64.b64encode(image_stream.getvalue()).decode('utf-8')
    return base64_image

    
@app.route('/get_graph_data')
def get_graph_data():
    # Return the current timestamp as a unique parameter to prevent caching
    timestamp = int(time.time() * 1000)
    return jsonify({'graph_src': f'static/graph.jpg?{timestamp}'})

@app.route('/quantum_job')
def quantum_job_route():
    if quantum_job.training_complete:
        quantum_job.run_quantum_job() 
        quantum_job.training_complete = False
        
        return jsonify({'result': 'success', 'progress': 0})
    else:
        return jsonify({'result': 'training_in_progress', 'progress': quantum_job.progress})


@app.route('/get_hist')
def get_hist():
     # Return the current timestamp as a unique parameter to prevent caching
    timestamp = int(time.time() * 4000)
    return jsonify({'hist_src': f'static/histogram.jpg?{timestamp}'})

    
  
  

    

if __name__ == '__main__':
    app.run(port=5887, debug=False, use_reloader=False)   
    
def connect_to_backend(backend_name):
    if backend_name == 'local':
        return Aer.get_backend('statevector_simulator')
    elif backend_name == 'ionq':
        provider = AzureQuantumProvider(
            resource_id='/subscriptions/your_subscription_id/resourceGroups/your_resource_group/providers/Microsoft.Quantum/Workspaces/your_workspace_id',
            location='your_location'
        )
        return provider.get_backend('ionq.qpu.h1')
    # Add more backends here if needed

if __name__ == '__main__':
    app.run(port=5044,debug=False,use_reloader=False)

SyntaxError: invalid syntax (3832776364.py, line 55)